In [1]:
import requests
import os 
import snowflake.connector
import time
import json
import flask

/Users/raph/Documents/rapmac-repo/.virtualenvs/scalability/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: dlopen(/Users/raph/Documents/rapmac-repo/.virtualenvs/scalability/lib/python3.9/site-packages/snowflake/connector/arrow_iterator.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '__ZN5arrow17Decimal128BuilderC1ERKNSt3__110shared_ptrINS_8DataTypeEEEPNS_10MemoryPoolE'


In [18]:
def getDomain(url):
    url = url.replace("https://www.", "").replace("http://www.", "").replace("www.","")
    return url

def querySFCompany(domain):
    return "select count(*) from dbt_db.models_final.final_companies_flatten where COMPANY_DOMAIN LIKE '" +domain+"'"

def scrappingURL1(domain):
    return "https://app.scrapingbee.com/api/v1/?api_key=MLINMYF4NZMFT3WA37AGIJ54AP10F86H8TDXSFNJSOIG3NRXMKXE04JSEWID422DT3SDOPEHPE2F3VKC&url=http://www."+domain+"&render_js=false&extract_rules={%22links%22:%20{%22selector%22:%20%22//a[contains(@href,%27linkedin.com/company%27)]%22,%20%22type%22:%20%22list%22,%20%22output%22:%20%22@href%22}}"

def scrappingURL2(domain):
    return "https://app.scrapingbee.com/api/v1/?api_key=MLINMYF4NZMFT3WA37AGIJ54AP10F86H8TDXSFNJSOIG3NRXMKXE04JSEWID422DT3SDOPEHPE2F3VKC&url=https%3A%2F%2Fwww.google.com%2Fsearch%3Fq%3Dsite%253Alinkedin.com%252Fcompany%2B"+domain+"&extract_rules=%7B%22first_link%22%3A%7B%22selector%22%3A%22%23search%20a%22%2C%22output%22%3A%22%40href%22%7D%7D&custom_google=True"

In [61]:
url="reveal.com"

In [26]:
url="reveal.com"
query = querySFCompany(getDomain(url))
DATABASE='dbt_db'
SCHEMA='models.final'
ctx = snowflake.connector.connect(
    user=os.environ['USER_SNOWFLAKE'],
    password=os.environ['PASSWORD_SNOWFLAKE'],
    account=os.environ['ACCOUNT_SNOWFLAKE'],
    warehouse='COMPUTE_WH',
    database=DATABASE,
    schema=SCHEMA
    )
cs = ctx.cursor()
cs.execute(query)
query_id = cs.sfqid
while ctx.is_still_running(ctx.get_query_status(query_id)):
  time.sleep(1)
cs.get_results_from_sfqid(query_id)
results = cs.fetchall()
print(f'{results[0]}')
ctx.close()

(0,)


In [45]:
x = requests.get(scrappingURL1(getDomain(url)))
print(x.text)
result = json.loads(x.text)

{"links": []}


In [46]:
print(result)
len(result)

{'links': []}


1

In [47]:
if len(result['links']) == 0:
    result = json.loads(requests.get(scrappingURL2(getDomain(url))).text)
    print(result['first_link'])
elif len(result['links']) != 0 :
    print(result['links'][0])

https://www.linkedin.com/company/reveal-group


In [48]:
url = "https://api.captaindata.co/v3/integrations/linkedin/accounts?page=0"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}

response = requests.get(url, headers=headers)
result= json.loads(response.text)
print(result)

{'integration': {'uid': 'linkedin', 'title': 'LinkedIn'}, 'accounts': [{'setup_at': '2023-02-06T16:23:23.114398', 'name': 'Victor Cariou', 'cookies': {'JSESSIONID': '"ajax:3377178882135116813"', 'li_a': None, 'li_at': 'AQEDASOu32kEADNMAAABhieJn4kAAAGGS5YjiVYAlbCNbWoTpTTM9B_qc6Et6ZdlPvM0tfkDsl3KeJavcaCxbALylTQCFgWf9qtZUC1m3JQ_jzX-vDSmOT9XZFFvh25P88sc8UrkzfJ24pOJ_FqO_rSw'}, 'via_extension': True, 'is_shared': True, 'level': 'Sales Navigator', 'is_valid': True, 'uid': 'eb0832aa-38d8-435f-8869-c4f53b00d747', 'application_permalink': 'linkedin', 'application_name': 'LinkedIn'}, {'setup_at': '2023-02-06T14:53:48.823359', 'name': 'Johann Lemena', 'cookies': {'JSESSIONID': '"ajax:3458321767039518452"', 'li_a': 'AQJ2PTEmc2FsZXNfY2lkPTQ5OTE2NDcwMiUzQSUzQTQ0NDIxMDAwN5l_w2kphKDfMQbZqRNsR_Cp7KB2', 'li_at': 'AQEDATLHmdQCGXV4AAABhhIru0QAAAGGNjg_RE4An5V_jUj8ccJEz2YvQzPj-Au-Q5f2JADCMXvsU4vXjM_pQRO4pIbvfx-E9KnYlJWLdtDNCU0iYbbeCZ5852iP7udccM-KsXx8WYp9zPFtCtLA0CpR'}, 'via_extension': True, 'is_shared': Tr

In [51]:
for i in range(len(result['accounts'])):
    if result['accounts'][i]['is_valid'] == True:
        uid = result['accounts'][i]['uid']
        print(result['integration'])
        print(result['accounts'][i]['name'])
#return 'Aucun compte LinkedIn n\'est actuellement valide sur Captain Data'
print('Aucun compte LinkedIn n\'est actuellement valide sur Captain Data')
#print(result['accounts'][0]['is_valid'])

eb0832aa-38d8-435f-8869-c4f53b00d747
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Cariou
efbb4db4-c60b-4ff5-b6dd-25560894c8aa
{'uid': 'linkedin', 'title': 'LinkedIn'}
Johann Lemena
a4960e9d-52d4-4b6f-8fff-b606034bc8ad
{'uid': 'linkedin', 'title': 'LinkedIn'}
Arthur Morizur
077bf979-0b37-4ed6-a349-d3dee297aaeb
{'uid': 'linkedin', 'title': 'LinkedIn'}
Victor Alexandrian
3dad7d0a-25d5-4dfb-8e16-690006957061
{'uid': 'linkedin', 'title': 'LinkedIn'}
Etienne Dubois
d9bfcd2c-25d5-477c-a77f-33c2082c3ea1
{'uid': 'linkedin', 'title': 'LinkedIn'}
Benjamin Donteville
Aucun compte LinkedIn n'est actuellement valide sur Captain Data


In [14]:
print(result['accounts'][0])

{'setup_at': '2023-02-06T16:23:23.114398', 'name': 'Victor Cariou', 'cookies': {'JSESSIONID': '"ajax:3377178882135116813"', 'li_a': None, 'li_at': 'AQEDASOu32kEADNMAAABhieJn4kAAAGGS5YjiVYAlbCNbWoTpTTM9B_qc6Et6ZdlPvM0tfkDsl3KeJavcaCxbALylTQCFgWf9qtZUC1m3JQ_jzX-vDSmOT9XZFFvh25P88sc8UrkzfJ24pOJ_FqO_rSw'}, 'via_extension': True, 'is_shared': True, 'level': 'Sales Navigator', 'is_valid': True, 'uid': 'eb0832aa-38d8-435f-8869-c4f53b00d747', 'application_permalink': 'linkedin', 'application_name': 'LinkedIn'}


In [56]:
linkedinUrl = 'https://www.linkedin.com/company/reveal-group'
uid = "eb0832aa-38d8-435f-8869-c4f53b00d747"
url = "https://api.captaindata.co/v3/workflows/b7aaec34-6e36-40b3-9f1f-376276d07af1/schedule"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}

body = {
    "steps": [
        {
            "accounts": [
                uid
            ],
            "parameters": {
                "extractOpenPositions": False
            },
            "output_column": None,
            "step_uid": "964f6c7c-9c73-4aff-a6e9-790f0c836565"
        }
    ],
    "inputs": [
        {
            "linkedin_company_url": linkedinUrl,
            "url": linkedinUrl
        }
    ],
    "job_name": uid
}

response = requests.post(url, headers=headers, data=json.dumps(body))
result= json.loads(response.text)
print(response.text)




{"template":null,"workflow":{"uid":"b7aaec34-6e36-40b3-9f1f-376276d07af1","permalink":"clay-extract-linkedin-company-profile3d301bbe-34fa-4514-9e30-a7cc2d446654","name":"CLAY - Extract LinkedIn Company Profile","created_at":"2023-02-03T14:37:00.586478","project_id":2774,"linked_templates":["8e315ab4-bce9-47ca-aaad-66defed46c61"],"template_uid":"8e315ab4-bce9-47ca-aaad-66defed46c61"},"message":"Bot successfully scheduled.","job_uid":"ac4b0a72-e543-4865-aac8-c552e177e4ba"}


In [60]:
print(result['job_uid'])

ac4b0a72-e543-4865-aac8-c552e177e4ba


In [54]:
url = "https://api.captaindata.co/v3/jobs/"+'19fb276a-f722-40ee-9295-60e648227aa9'+"/results"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}
response = requests.get(url, headers=headers)

In [55]:
print(response.text)

{"items_count":1,"pages":1,"limit":1000,"results":[{"updated_at":"2023-02-09T10:15:24.570000","company_name":"Reveal Group","name":"Reveal Group","specialties":["Operations Management","Workforce Management","Robotic Process Automation","Performance Management","Change Management","Enterprise Transformation","Business Process Management (BPM) Software","Business Process Automation Software","Intelligent Automation"],"specialities":["Operations Management","Workforce Management","Robotic Process Automation","Performance Management","Change Management","Enterprise Transformation","Business Process Management (BPM) Software","Business Process Automation Software","Intelligent Automation"],"tagline":"We help organizations #AutomateBetter. Proud UiPath USN Partner + only Blue Prism Double Platinum Partner. ","linkedin_company_id":"77340","id":"77340","description":"Reveal Group is your Intelligent Automation services company. We have been helping clients to Automate Better since 2005. Revea

In [62]:
job_uid = 'ab41a35a-2c5c-4143-ac71-675406bfeebc'
url = "https://api.captaindata.co/v3/jobs/"+job_uid+"/results"

headers = {
    "Content-Type": "application/json",
    "x-project-id": "24dc2e2c-d2e6-433e-81d2-2f87bc87b59d",
    "Authorization": "x-api-key feecb369cafafe86a22255bca8b6293a4c7a2281bd751d453f06693af32f458d"
}
response = requests.get(url, headers=headers)

In [64]:
print(response.text)

{"items_count":0,"pages":0,"limit":1000,"results":[],"paging":{"previous":null,"next":null,"have_next_page":false}}
